In [57]:
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

# Load all JSON files from the tournaments_data folder
data_folder = "tournaments_data"
all_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.json')]

# Initialize lists to store extracted data
tournament_info_list = []
team_members_list = []
match_cards_list = []
maps_list = []

# Helper function to extract map data
def extract_map_data(match_id, match_data):
    maps = []
    for key, value in match_data.items():
        if key.startswith('map'):
            map_num = key[3:]
            map_info = value if isinstance(value, dict) else {}
            map_info['map_num'] = map_num
            map_info['match_card_id'] = match_id
            maps.append(map_info)
    return maps

# ID counters
tournament_id_counter = 1
team_id_counter = 1
match_id_counter = 1

# Process each file with a progress bar
for file in tqdm(all_files, desc="Processing JSON files"):
    with open(file, 'r') as f:
        data = json.load(f)
        
        # Extract tournament_info
        tournament_info = data.get("tournament_info", [])
        if tournament_info:
            tournament_info = tournament_info[0]
        else:
            tournament_info = {}
        
        tournament_info["tournament_id"] = tournament_id_counter
        tournament_info_list.append(tournament_info)
        
        # Extract team members
        team_cards = data.get("team_cards", [])
        for team in team_cards:
            if not team.get("team"):
                continue
            team_id = team_id_counter
            team_id_counter += 1
            members = team.get("members", [])
            for member in members:
                if not member.get("name"):
                    continue
                team_member_info = {
                    "team": team.get("team", ""),
                    "tournament_id": tournament_id_counter,
                    "team_id": team_id
                }
                team_member_info.update(member)  # Add all keys from member
                team_members_list.append(team_member_info)
        
        # Extract match cards
        match_cards = data.get("match_cards", [])
        for match in match_cards:
            if not match.get("winner"):
                continue
            match_id = match_id_counter
            match_id_counter += 1
            match['tournament_id'] = tournament_id_counter
            match['match_card_id'] = match_id
            match_cards_list.append(match)
            
            # Extract maps data from match cards
            maps = extract_map_data(match_id, match)
            maps_list.extend(maps)
    
    tournament_id_counter += 1

# Convert lists to DataFrames
tournament_info_df = pd.DataFrame(tournament_info_list)
team_members_df = pd.DataFrame(team_members_list)
match_cards_df = pd.DataFrame(match_cards_list)
maps_df = pd.DataFrame(maps_list)

Processing JSON files: 100%|██████████| 3685/3685 [00:00<00:00, 3973.92it/s]


In [58]:
# Convert all empty whitespace values to NaN
def convert_whitespace_to_nan(df):
    return df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == '' else x)

tournament_info_df = convert_whitespace_to_nan(tournament_info_df)
team_members_df = convert_whitespace_to_nan(team_members_df)
match_cards_df = convert_whitespace_to_nan(match_cards_df)
maps_df = convert_whitespace_to_nan(maps_df)

# Function to validate date
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str, format="%Y-%m-%d", errors='raise')
        return True
    except ValueError:
        return False

# Function to validate date and convert to datetime
def to_datetime(date_str):
    try:
        return pd.to_datetime(date_str, format="%Y-%m-%d", errors='raise')
    except ValueError:
        return pd.NaT
    
# Function to compare dates and return the least recent valid date
def get_least_recent_date(dates):
    valid_dates = [date for date in dates if pd.notna(to_datetime(date))]
    return min(valid_dates) if valid_dates else None

# Calculate agg_date
tournament_info_df['agg_date'] = tournament_info_df.apply(lambda row: get_least_recent_date(
    [row.get('date', ''), row.get('sdate', ''), row.get('edate', '')]
), axis=1)

# Convert agg_date to just the date part (YYYY-MM-DD)
#tournament_info_df['agg_date'] = pd.to_datetime(tournament_info_df['agg_date']).dt.date

# Drop rows with no valid date
tournament_info_df = tournament_info_df[tournament_info_df['agg_date'].notna()]

# Sort by agg_date and create the date_order column
tournament_info_df = tournament_info_df.sort_values(by='agg_date').reset_index(drop=True)
tournament_info_df['date_order'] = tournament_info_df.index + 1

# Drop rows where liquipediatier column is 5.0
tournament_info_df = tournament_info_df[tournament_info_df['liquipediatier'] != 5.0]

# Keep only specified columns
columns_to_keep = ['name', 'series', 'organizer', 'type', 'city', 'country', 'prizepool', 'prizepoolusd', 'format', 'date', 'sdate', 'edate', 'liquipediatier', 'team_number', 'agg_date', 'tournament_id', 'date_order']
tournament_info_df = tournament_info_df[columns_to_keep]

# Process match_cards_df
# Fill missing date values from tournament agg_date
match_cards_df = match_cards_df.merge(tournament_info_df[['tournament_id', 'agg_date']], on='tournament_id', how='left', suffixes=('', '_tournament'))
match_cards_df['date'] = match_cards_df.apply(lambda row: row['date'] if not pd.isna(row['date']) else row['agg_date'], axis=1)
match_cards_df.drop(columns=['agg_date'], inplace=True)

# Drop rows without opponent1 or opponent2
match_cards_df = match_cards_df.dropna(subset=['opponent1', 'opponent2'])

# Drop rows where opponent1 or opponent2 is 'BYE'
match_cards_df = match_cards_df[
    (match_cards_df['opponent1'] != 'BYE') & (match_cards_df['opponent2'] != 'BYE')
]

# Keep only specified columns
columns_to_keep = ['date', 'date_timezone', 'opponent1', 'opponent2', 'opponent1_score', 'opponent2_score', 'winner', 'format', 'date_time', 'key', 'tournament_id', 'match_card_id']
match_cards_df = match_cards_df[columns_to_keep]

# Keep only specified columns
columns_to_keep = ['map', 'mode', 'score1', 'score2', 'winner', 'map_num', 'match_card_id'] 
maps_df = maps_df[columns_to_keep]

# Drop related entries in other dataframes for removed tournaments
valid_tournament_ids = set(tournament_info_df['tournament_id'])

# Drop match_cards_df records where tournament_id is not in tournament_id in tournament_info_df
match_cards_df = match_cards_df[match_cards_df['tournament_id'].isin(valid_tournament_ids)]

# Drop maps where match_card_id is not in match_cards_df.match_card_id
valid_match_card_ids = set(match_cards_df['match_card_id'])
maps_df = maps_df[maps_df['match_card_id'].isin(valid_match_card_ids)]

# Drop teams where tournament_id is not in tournament_info_df
team_members_df = team_members_df[team_members_df['tournament_id'].isin(valid_tournament_ids)]

# Track if each tournament has players or not for later.
tournament_info_df['has_players'] = tournament_info_df['tournament_id'].isin(team_members_df['tournament_id'])


C:\Users\jorda\AppData\Local\Temp\ipykernel_17040\1363133916.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == '' else x)


In [59]:
import pandas as pd
from tqdm import tqdm
from difflib import get_close_matches
import re
import math
import json

# Set up tqdm with pandas
tqdm.pandas()

# Load the acronym map
with open('acronym_map.json', 'r') as file:
    acronym_map = json.load(file)

# Map opponent acronyms to full names, retain original name if not found
def map_acronyms(team_name):
    return acronym_map.get(team_name, team_name)

match_cards_df['opponent1'] = match_cards_df['opponent1'].apply(map_acronyms)
match_cards_df['opponent2'] = match_cards_df['opponent2'].apply(map_acronyms)

# Terms to remove
terms_to_remove = ['GAMING', 'TEAM', 'ESPORTS', 'ESPORT', 'CLUB']

def clean_team_name(name):
    # Remove specified terms
    for term in terms_to_remove:
        name = name.replace(term, '')
    # Remove non-alphanumeric characters
    name = re.sub(r'\W+', '', name)
    return name.strip().upper()  # Convert to upper case for uniformity

# Create clean_team column in team_members_df
team_members_df['clean_team'] = team_members_df['team'].apply(clean_team_name)
tournament_id_to_date_order = tournament_info_df.set_index('tournament_id')['date_order'].to_dict()
tournament_id_to_liquipediatier = tournament_info_df.set_index('tournament_id')['liquipediatier'].to_dict()
tournament_id_to_agg_date = tournament_info_df.set_index('tournament_id')['agg_date'].to_dict()

# Add date_order, liquipediatier, and agg_date to match_cards_df using the mapping
match_cards_df['date_order'] = match_cards_df['tournament_id'].map(tournament_id_to_date_order)
match_cards_df['liquipediatier'] = match_cards_df['tournament_id'].map(tournament_id_to_liquipediatier)
match_cards_df['agg_date'] = match_cards_df['tournament_id'].map(tournament_id_to_agg_date)

# Create unique_teams dataframe directly from team_members_df
unique_teams = team_members_df[['clean_team', 'team_id', 'tournament_id']].drop_duplicates()

# Add date_order, liquipediatier, and agg_date to unique_teams using the mapping
unique_teams['date_order'] = unique_teams['tournament_id'].map(tournament_id_to_date_order)
unique_teams['liquipediatier'] = unique_teams['tournament_id'].map(tournament_id_to_liquipediatier)
unique_teams['agg_date'] = unique_teams['tournament_id'].map(tournament_id_to_agg_date)

# Initial matching within the same tournament
def find_closest_team_id_within_tournament(row, unique_teams, threshold=0.8):
    tournament_teams = unique_teams[unique_teams['tournament_id'] == row['tournament_id']]
    team_names = tournament_teams['clean_team'].tolist()
    team_ids = tournament_teams['team_id'].tolist()
    
    best_team1_id, best_team2_id = None, None
    
    if team_names:
        matches1 = get_close_matches(row['clean_opponent1'], team_names, n=1, cutoff=threshold)
        matches2 = get_close_matches(row['clean_opponent2'], team_names, n=1, cutoff=threshold)
        
        if matches1:
            best_team1_id = team_ids[team_names.index(matches1[0])]
        if matches2:
            best_team2_id = team_ids[team_names.index(matches2[0])]
    
    return best_team1_id, best_team2_id

# Apply initial matching with progress bar
match_cards_df['clean_opponent1'] = match_cards_df['opponent1'].apply(clean_team_name)
match_cards_df['clean_opponent2'] = match_cards_df['opponent2'].apply(clean_team_name)

# Initial matching
team_ids = match_cards_df.progress_apply(lambda row: find_closest_team_id_within_tournament(row, unique_teams, 0.8), axis=1)
match_cards_df[['team1_id', 'team2_id']] = pd.DataFrame(team_ids.tolist(), index=match_cards_df.index)

# Function to find the closest matching team_id with index distance, liquipediatier constraint, and identical agg_date check
def find_closest_team_id_with_index_distance(row, unique_teams, threshold=0.8, max_index=1):
    date_order = row['date_order']
    agg_date = row['agg_date']
    tier = row['liquipediatier']
    date_range = range(date_order - max_index, date_order + 1)  # Looking backward in time
    
    candidate_teams = unique_teams[((unique_teams['date_order'].isin(date_range)) | 
                                   (unique_teams['agg_date'] == agg_date)) & 
                                   (unique_teams['liquipediatier'].between(tier - 1, tier + 1))]
    team_names = candidate_teams['clean_team'].tolist()
    team_ids = candidate_teams['team_id'].tolist()
    
    best_team1_id, best_team2_id = None, None
    
    if team_names:
        matches1 = get_close_matches(row['clean_opponent1'], team_names, n=1, cutoff=threshold)
        matches2 = get_close_matches(row['clean_opponent2'], team_names, n=1, cutoff=threshold)
        
        if matches1:
            best_team1_id = team_ids[team_names.index(matches1[0])]
        if matches2:
            best_team2_id = team_ids[team_names.index(matches2[0])]
    
    return best_team1_id, best_team2_id

# Loop to progressively reduce threshold and increase index distance
def progressively_match_teams(match_cards_df, unique_teams):
    threshold = 1
    max_index = 1

    while True:
        missing_team1_ids = match_cards_df['team1_id'].isna()
        missing_team2_ids = match_cards_df['team2_id'].isna()

        if not missing_team1_ids.any() and not missing_team2_ids.any():
            break

        print(f"Threshold {threshold}. Max Index {max_index}")

        team_ids = match_cards_df[missing_team1_ids | missing_team2_ids].progress_apply(
            lambda row: find_closest_team_id_with_index_distance(row, unique_teams, threshold, max_index),
            axis=1
        )
        
        match_cards_df.loc[missing_team1_ids, 'team1_id'] = team_ids.apply(lambda x: x[0])
        match_cards_df.loc[missing_team2_ids, 'team2_id'] = team_ids.apply(lambda x: x[1])

        # Decrease threshold and increase index distance
        threshold -= 0.02
        max_index = math.ceil(max_index * 1.5)

        # Stop if threshold is too low
        if threshold < 0.7:
            break

# Apply the progressive matching
progressively_match_teams(match_cards_df, unique_teams)


100%|██████████| 26745/26745 [00:05<00:00, 4582.41it/s]


Threshold 1. Max Index 1


100%|██████████| 12651/12651 [00:12<00:00, 1008.79it/s]


Threshold 0.98. Max Index 2


100%|██████████| 10172/10172 [00:10<00:00, 986.24it/s] 


...

100%|██████████| 7611/7611 [00:16<00:00, 449.53it/s]


Threshold 0.7399999999999998. Max Index 315


100%|██████████| 7249/7249 [00:36<00:00, 199.40it/s]


Threshold 0.7199999999999998. Max Index 473


100%|██████████| 6863/6863 [00:28<00:00, 241.00it/s]


In [60]:
# Function to duplicate team members and assign new team IDs
def duplicate_team_members(team_members_df, original_team_id, new_team_id):
    new_team_members = team_members_df[team_members_df['team_id'] == original_team_id].copy()
    new_team_members['team_id'] = new_team_id
    return new_team_members

# Create a new DataFrame to store duplicated team members
duplicated_team_members = pd.DataFrame()

# Keep track of the original team IDs that have been duplicated
duplicated_team_ids = set()

# Get the maximum existing team ID and initialize new_team_id counter
max_team_id = team_members_df['team_id'].max()
new_team_id_counter = max_team_id + 1

# Loop through match_cards_df to create unique team IDs and duplicate team members
for index, row in tqdm(match_cards_df.iterrows(), total=match_cards_df.shape[0]):
    if not pd.isna(row['team1_id']):
        new_team1_id = new_team_id_counter
        duplicated_team_members = pd.concat([duplicated_team_members, duplicate_team_members(team_members_df, row['team1_id'], new_team1_id)])
        duplicated_team_ids.add(row['team1_id'])
        match_cards_df.at[index, 'team1_id'] = new_team1_id
        new_team_id_counter += 1
    
    if not pd.isna(row['team2_id']):
        new_team2_id = new_team_id_counter
        duplicated_team_members = pd.concat([duplicated_team_members, duplicate_team_members(team_members_df, row['team2_id'], new_team2_id)])
        duplicated_team_ids.add(row['team2_id'])
        match_cards_df.at[index, 'team2_id'] = new_team2_id
        new_team_id_counter += 1

  0%|          | 0/26745 [00:00<?, ?it/s]

100%|██████████| 26745/26745 [05:26<00:00, 81.86it/s] 


In [61]:
team_members_df = duplicated_team_members

# Identify valid team IDs based on the presence of players
valid_team_ids = set(team_members_df[team_members_df['role_type'] == 'player']['team_id'].unique())

# Set team1_id and team2_id to null if they do not point to any team members with players
match_cards_df.loc[~match_cards_df['team1_id'].isin(valid_team_ids), 'team1_id'] = None
match_cards_df.loc[~match_cards_df['team2_id'].isin(valid_team_ids), 'team2_id'] = None

# Drop match_cards that have null in either team1_id or team2_id
match_cards_df.dropna(subset=['team1_id', 'team2_id'], inplace=True)

# Filter out team members that are not referenced in match_cards_df
used_team_ids = set(match_cards_df['team1_id']).union(set(match_cards_df['team2_id']))
team_members_df = team_members_df[team_members_df['team_id'].isin(used_team_ids)]

In [62]:
from collections import defaultdict

# Display the initial info of the DataFrame
team_members_df.info()

# Helper function to find the most common value in a list
def most_common(lst):
    return max(set(lst), key=lst.count)

# Helper function to create lookup dictionary
def create_lookup(df, key_cols, value_col):
    lookup = defaultdict(list)
    for _, row in df[df[value_col].notna()].iterrows():
        key = tuple(row[col] for col in key_cols)
        lookup[key].append(row[value_col])
    return {
        key: most_common(values)
        for key, values in lookup.items()
    }

# Impute missing values based on lookup dictionary
def impute_missing_values(df, lookup, key_cols, value_col):
    for idx, row in df[df[value_col].isna()].iterrows():
        key = tuple(row[col] for col in key_cols)
        if key in lookup:
            df.at[idx, value_col] = lookup[key]

# Step 2: Impute missing position values
position_lookup = create_lookup(team_members_df, ["name", "flag", "role_type"], "position")
impute_missing_values(team_members_df, position_lookup, ["name", "flag", "role_type"], "position")

# Step 3: Impute missing flag values
flag_lookup = create_lookup(team_members_df, ["name", "position", "role_type"], "flag")
impute_missing_values(team_members_df, flag_lookup, ["name", "position", "role_type"], "flag")

# Step 4: Impute remaining values based on name and role_type alone
position_lookup_name = create_lookup(team_members_df, ["name", "role_type"], "position")
impute_missing_values(team_members_df, position_lookup_name, ["name", "role_type"], "position")

flag_lookup_name = create_lookup(team_members_df, ["name", "role_type"], "flag")
impute_missing_values(team_members_df, flag_lookup_name, ["name", "role_type"], "flag")

# Step 6: Create a unique ID for combinations of NAME, FLAG, POSITION, and ROLE_TYPE
combination_to_id = {}
current_id = 1

def get_combination_id(row):
    global current_id
    if row["role_type"] == 'player':
        key = (
            row["name"],
            row["flag"] if pd.notna(row["flag"]) else None,
            row["position"] if pd.notna(row["position"]) else None
        )
    else:
        key = (
            row["name"],
            row["flag"] if pd.notna(row["flag"]) else None,
            row["role_type"]
        )
        
    if key not in combination_to_id:
        combination_to_id[key] = current_id
        current_id += 1
        
    return combination_to_id[key]

team_members_df["unique_id"] = team_members_df.apply(get_combination_id, axis=1)

# Display the updated DataFrame info
team_members_df.info()




<class 'pandas.core.frame.DataFrame'>
Index: 317206 entries, 6 to 200424
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   team           317206 non-null  object
 1   tournament_id  317206 non-null  int64 
 2   team_id        317206 non-null  int64 
 3   name           317206 non-null  object
 4   position       250368 non-null  object
 5   flag           171400 non-null  object
 6   role_type      317206 non-null  object
 7   clean_team     317206 non-null  object
dtypes: int64(2), object(6)
memory usage: 21.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 317206 entries, 6 to 200424
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   team           317206 non-null  object
 1   tournament_id  317206 non-null  int64 
 2   team_id        317206 non-null  int64 
 3   name           317206 non-null  object
 4   position       280048 non-null

C:\Users\jorda\AppData\Local\Temp\ipykernel_17040\957915810.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_members_df["unique_id"] = team_members_df.apply(get_combination_id, axis=1)


In [63]:
import pandas as pd
from tqdm import tqdm

# Configuration parameters
config = {
    "base_k": 50,  # Base K-factor for ELO calculations
    "initial_elo": 1500.0,  # Initial ELO rating for all players and coaches
    "tier_multiplier_factor_pre_2024": 12,  # Pre-2024 tier multiplier factor
    "tier_exponent_pre_2024": 2,  # Pre-2024 tier exponent
    "tier_multiplier_factor_post_2024": 6,  # Post-2024 tier multiplier factor
    "tier_exponent_post_2024": 1,  # Post-2024 tier exponent
}

# There will be duplicate indexes from data duplication step. We use indexes to speed up this step 4x.
team_members_df.reset_index(drop=True, inplace=True)

def calculate_elo_change_common(player_avg_elo, opponent_avg_elo, outcome, base_k, tier_multiplier):
    expected_score = 1 / (1 + 10 ** ((opponent_avg_elo - player_avg_elo) / 400))
    change = (base_k * tier_multiplier) * (outcome - expected_score)
    return change, expected_score

def create_team_member_lookup(members_df):
    # Create a dictionary to map team_id to player and coach indices
    team_member_lookup = {}
    
    for idx, row in members_df.iterrows():
        team_id = row["team_id"]
        role_type = row["role_type"]
        position = row.get("position", "")

        if team_id not in team_member_lookup:
            team_member_lookup[team_id] = {
                "players": [],
                "all_members": []
            }
        
        if role_type == 'player':
            team_member_lookup[team_id]["players"].append(idx)
        if role_type == 'player' or (pd.notna(position) and 'COACH' in position):
            team_member_lookup[team_id]["all_members"].append(idx)
    
    return team_member_lookup

def update_elo_ratings(matches_df, members_df, config):
    base_k = config["base_k"]
    initial_elo = config["initial_elo"]
    tier_multiplier_factor_pre_2024 = config["tier_multiplier_factor_pre_2024"]
    tier_exponent_pre_2024 = config["tier_exponent_pre_2024"]
    tier_multiplier_factor_post_2024 = config["tier_multiplier_factor_post_2024"]
    tier_exponent_post_2024 = config["tier_exponent_post_2024"]

    # Sort data outside the loop
    matches_df.sort_values(by='date', inplace=True)

    # Initialize ELO ratings
    members_df['elo_before'] = initial_elo
    members_df['elo_after'] = initial_elo
    members_df['c_elo_before'] = initial_elo
    members_df['c_elo_after'] = initial_elo
    elo_ratings = {}
    coach_elo_ratings = {}

    # Drop columns if they already exist
    for col in ['team1_avg_elo', 'team2_avg_elo', 'team1_expected_outcome', 'team2_expected_outcome']:
        if col in matches_df.columns:
            matches_df.drop(columns=[col], inplace=True)

    # Initial values for tier multipliers
    tier_multiplier_factor = tier_multiplier_factor_pre_2024
    tier_exponent = tier_exponent_pre_2024

    # Create a lookup table for team members
    team_member_lookup = create_team_member_lookup(members_df)

    for _, match in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Processing matches"):
        # Pre-fetch match values
        match_id = match["match_card_id"]
        date = match["date"]
        winner = match["winner"]
        team1_id = match["team1_id"]
        team2_id = match["team2_id"]
        tier = match["liquipediatier"]

        # Ensure both team1_id and team2_id are present
        if pd.isna(team1_id) or pd.isna(team2_id):
            continue
        
        # Get team player indices once and reuse
        team1_player_indices = team_member_lookup.get(team1_id, {}).get("players", [])
        team2_player_indices = team_member_lookup.get(team2_id, {}).get("players", [])
        
        # Get all member indices (players and staff with 'COACH' in their position)
        team1_all_member_indices = team_member_lookup.get(team1_id, {}).get("all_members", [])
        team2_all_member_indices = team_member_lookup.get(team2_id, {}).get("all_members", [])

        # Skip matches where either team has no players
        if not team1_player_indices or not team2_player_indices:
            continue

        # Update tier multipliers if the date threshold is crossed
        if date >= "2024-01-01" and (tier_multiplier_factor != tier_multiplier_factor_post_2024 or tier_exponent != tier_exponent_post_2024):
            tier_multiplier_factor = tier_multiplier_factor_post_2024
            tier_exponent = tier_exponent_post_2024
        
        if winner not in [1, 2]:
            continue
        
        tier_multiplier = tier_multiplier_factor / ((tier + 1) ** tier_exponent)
        
        # Retrieve and set ELOs for team members before the match
        for idx in team1_player_indices + team2_player_indices:
            unique_id = members_df.at[idx, 'unique_id']
            if unique_id not in elo_ratings:
                elo_ratings[unique_id] = initial_elo
        
        for idx in team1_all_member_indices + team2_all_member_indices:
            unique_id = members_df.at[idx, 'unique_id']
            if unique_id not in coach_elo_ratings:
                coach_elo_ratings[unique_id] = initial_elo

        team1_avg_elo = sum(elo_ratings[members_df.at[idx, 'unique_id']] for idx in team1_player_indices) / len(team1_player_indices)
        team2_avg_elo = sum(elo_ratings[members_df.at[idx, 'unique_id']] for idx in team2_player_indices) / len(team2_player_indices)
        team1_avg_coach_elo = sum(coach_elo_ratings[members_df.at[idx, 'unique_id']] for idx in team1_all_member_indices) / len(team1_all_member_indices)
        team2_avg_coach_elo = sum(coach_elo_ratings[members_df.at[idx, 'unique_id']] for idx in team2_all_member_indices) / len(team2_all_member_indices)

        outcome1 = 1 if winner == 1 else 0

        team1_common_change, team1_expected_outcome = calculate_elo_change_common(team1_avg_elo, team2_avg_elo, outcome1, base_k, tier_multiplier)
        team2_common_change = -team1_common_change  # The change for team 2 is the inverse of team 1

        team1_coach_change, _ = calculate_elo_change_common(team1_avg_coach_elo, team2_avg_coach_elo, outcome1, base_k, tier_multiplier)
        team2_coach_change = -team1_coach_change  # The change for team 2 is the inverse of team 1

        # Store the average ELOs and expected outcomes in matches_df
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team1_avg_elo'] = team1_avg_elo
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team2_avg_elo'] = team2_avg_elo
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team1_expected_outcome'] = team1_expected_outcome
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team2_expected_outcome'] = 1 - team1_expected_outcome

        # Update ELOs for players and coaches using precomputed indices
        for team, team_common_change, team_coach_change, team_player_indices, team_all_member_indices in [
            (team1_id, team1_common_change, team1_coach_change, team1_player_indices, team1_all_member_indices),
            (team2_id, team2_common_change, team2_coach_change, team2_player_indices, team2_all_member_indices)
        ]:
            for idx in team_player_indices:
                unique_id = members_df.at[idx, 'unique_id']
                player_elo_before = elo_ratings[unique_id]
                player_elo_after = player_elo_before + team_common_change
                elo_ratings[unique_id] = player_elo_after

                # Update the elo_before and elo_after columns for each match
                members_df.at[idx, 'elo_before'] = player_elo_before
                members_df.at[idx, 'elo_after'] = player_elo_after

            for idx in team_all_member_indices:
                unique_id = members_df.at[idx, 'unique_id']
                coach_elo_before = coach_elo_ratings[unique_id]
                coach_elo_after = coach_elo_before + team_coach_change
                coach_elo_ratings[unique_id] = coach_elo_after

                # Update the c_elo_before and c_elo_after columns for each match
                members_df.at[idx, 'c_elo_before'] = coach_elo_before
                members_df.at[idx, 'c_elo_after'] = coach_elo_after

# Usage example with match_cards_df, team_members_df, and tournament_info_df already loaded
update_elo_ratings(match_cards_df, team_members_df, config)


C:\Users\jorda\AppData\Local\Temp\ipykernel_17040\4163476928.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  members_df['elo_before'] = initial_elo
C:\Users\jorda\AppData\Local\Temp\ipykernel_17040\4163476928.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  members_df['elo_after'] = initial_elo
C:\Users\jorda\AppData\Local\Temp\ipykernel_17040\4163476928.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [64]:
import pandas as pd
from fuzzywuzzy import process

# Assuming you have a DataFrame 'maps_df' with your data

# Step 1: Define valid map and mode combinations
valid_combinations = {
    "KING'S ROW": 'HYBRID',
    'NEPAL': 'CONTROL',
    'DORADO': 'ESCORT',
    'LIJIANG TOWER': 'CONTROL',
    'WATCHPOINT: GIBRALTAR': 'ESCORT',
    'NUMBANI': 'HYBRID',
    'HANAMURA': 'ASSAULT',
    'ILIOS': 'CONTROL',
    'TEMPLE OF ANUBIS': 'ASSAULT',
    'HOLLYWOOD': 'HYBRID',
    'NEW QUEEN STREET': 'PUSH',
    'ESPERANÇA': 'PUSH',
    'CIRCUIT ROYAL': 'ESCORT',
    'ROUTE 66': 'ESCORT',
    'EICHENWALDE': 'HYBRID',
    'VOLSKAYA INDUSTRIES': 'ASSAULT',
    'OASIS': 'CONTROL',
    'JUNKERTOWN': 'ESCORT',
    'HORIZON LUNAR COLONY': 'ASSAULT',
    'COLOSSEO': 'PUSH',
    'SURAVASA': 'FLASHPOINT',
    'MIDTOWN': 'HYBRID',
    'ANTARCTIC PENINSULA': 'CONTROL',
    'NEW JUNK CITY': 'FLASHPOINT',
    'SAMOA': 'CONTROL',
    'RIALTO': 'ESCORT',
    'BLIZZARD WORLD': 'HYBRID',
    'HAVANA': 'ESCORT',
    'BUSAN': 'CONTROL',
    'SHAMBALI MONASTERY': 'ESCORT',
    'PARAÍSO': 'HYBRID',
    'PARIS': 'ASSAULT'
}

# Convert the valid combinations dictionary to DataFrames for easier manipulation
valid_maps_df = pd.DataFrame(valid_combinations.keys(), columns=['map'])
valid_modes_df = pd.DataFrame(valid_combinations.values(), columns=['mode'])

# Step 2: Drop rows where 'map' or 'mode' is NaN
maps_df.dropna(subset=['map', 'mode'], inplace=True)

# Step 3: Fuzzy match invalid map names to the closest valid map names
def match_map(name, valid_names):
    result = process.extractOne(name, valid_names)
    return result[0] if result and result[1] >= 60 else name

maps_df['map'] = maps_df['map'].apply(lambda x: match_map(x, valid_maps_df['map']))

# Step 4: Fuzzy match invalid mode names to the closest valid mode names
def match_mode(name, valid_names):
    result = process.extractOne(name, valid_names)
    return result[0] if result and result[1] >= 60 else name

maps_df['mode'] = maps_df['mode'].apply(lambda x: match_mode(x, valid_modes_df['mode']))

# Step 5: Filter based on valid combinations
valid_combinations_df = pd.DataFrame(list(valid_combinations.items()), columns=['map', 'mode'])
maps_df = maps_df.merge(valid_combinations_df, on=['map', 'mode'], how='inner')

# Display the cleaned DataFrame
print(maps_df)


                 map     mode  score1  score2  winner map_num  match_card_id
0         KING'S ROW   HYBRID     3.0     4.0     2.0       1             16
1              NEPAL  CONTROL     3.0     0.0     1.0       2             16
2             DORADO   ESCORT     2.0     3.0     2.0       3             16
3         KING'S ROW   HYBRID     3.0     0.0     1.0       1             17
4      LIJIANG TOWER  CONTROL     3.0     1.0     1.0       2             17
...              ...      ...     ...     ...     ...     ...            ...
14744     KING'S ROW   HYBRID     1.0     2.0     2.0       1          47257
14745       ROUTE 66   ESCORT     1.0     2.0     2.0       2          47257
14746  LIJIANG TOWER  CONTROL     0.0     2.0     2.0       1          47258
14747     KING'S ROW   HYBRID     0.0     1.0     2.0       2          47258
14748  CIRCUIT ROYAL   ESCORT     0.0     1.0     2.0       3          47258

[14749 rows x 7 columns]


In [65]:
# Define the function to determine Overwatch version
def determine_overwatch_version(row, key):
    if row[key] >= '2022-10-04':
        return 2
    elif row['liquipediatier'] == 1 and row[key] >= '2022-05-05':
        return 2
    else:
        return 1

# Apply the function to both DataFrames
tournament_info_df['version'] = tournament_info_df.apply(determine_overwatch_version, axis=1, key='agg_date')
match_cards_df['version'] = match_cards_df.apply(determine_overwatch_version, axis=1, key='date')


In [66]:
# Import necessary libraries
from collections import Counter

# Create a dictionary to store the most common mode for each map
most_common_mode = {}

# Group by map and find the most common mode for each map
map_mode_groups = maps_df.groupby('map')['mode'].apply(lambda x: Counter(x).most_common(1)[0][0])

# Update the most_common_mode dictionary with the result
most_common_mode.update(map_mode_groups.to_dict())

# Apply the most common mode to each map in the DataFrame
maps_df['mode'] = maps_df['map'].map(most_common_mode)

# Display the cleaned DataFrame
print(maps_df)


                 map     mode  score1  score2  winner map_num  match_card_id
0         KING'S ROW   HYBRID     3.0     4.0     2.0       1             16
1              NEPAL  CONTROL     3.0     0.0     1.0       2             16
2             DORADO   ESCORT     2.0     3.0     2.0       3             16
3         KING'S ROW   HYBRID     3.0     0.0     1.0       1             17
4      LIJIANG TOWER  CONTROL     3.0     1.0     1.0       2             17
...              ...      ...     ...     ...     ...     ...            ...
14744     KING'S ROW   HYBRID     1.0     2.0     2.0       1          47257
14745       ROUTE 66   ESCORT     1.0     2.0     2.0       2          47257
14746  LIJIANG TOWER  CONTROL     0.0     2.0     2.0       1          47258
14747     KING'S ROW   HYBRID     0.0     1.0     2.0       2          47258
14748  CIRCUIT ROYAL   ESCORT     0.0     1.0     2.0       3          47258

[14749 rows x 7 columns]


In [67]:
import pandas as pd
from tqdm import tqdm

# Initialize variables
next_core_id = 1
core_rosters = {}

# Function to find a matching core with 60% similarity
def find_matching_core(new_roster, core_rosters):
    for core_id, roster in core_rosters.items():
        intersection = set(new_roster).intersection(set(roster))
        if len(intersection) / len(roster) > 0.6:
            return core_id
    return None

# Create a new column for core_id
team_members_df['core_id'] = None

# Sort match_cards_df by match_date
match_cards_df = match_cards_df.sort_values(by='date')

# Create a lookup table for team players
team_player_lookup = {}
for idx, row in team_members_df[team_members_df['role_type'] == 'player'].iterrows():
    team_id = row["team_id"]
    unique_id = row["unique_id"]

    if team_id not in team_player_lookup:
        team_player_lookup[team_id] = []

    team_player_lookup[team_id].append((unique_id, idx))

# Process each match in order with tqdm progress bar
for _, match_row in tqdm(match_cards_df.iterrows(), total=match_cards_df.shape[0], desc="Processing matches"):
    team_ids = [match_row['team1_id'], match_row['team2_id']]
    
    for team_id in team_ids:
        if pd.isna(team_id) or team_id not in team_player_lookup:
            continue
        
        # Get the set of players for the current team_id
        current_roster = [uid for uid, idx in team_player_lookup[team_id]]

        if len(current_roster) == 0:
            continue  # Skip if the roster is empty

        # Find a matching core
        core_id = find_matching_core(current_roster, core_rosters)
        
        if core_id is None:
            # If no matching core is found, create a new core_id
            core_id = next_core_id
            next_core_id += 1
            core_rosters[core_id] = current_roster
        
        # Assign the core_id to the current team's players
        for _, idx in team_player_lookup[team_id]:
            team_members_df.at[idx, 'core_id'] = core_id


C:\Users\jorda\AppData\Local\Temp\ipykernel_17040\1015975562.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_members_df['core_id'] = None
Processing matches: 100%|██████████| 20339/20339 [01:21<00:00, 248.55it/s]


In [68]:
# Convert the core_id column to integer
#Wteam_members_df['core_id'] = team_members_df['core_id'].astype(int)

# Group by core_id to count the number of matches each core has played
core_match_counts = team_members_df.groupby('core_id').size().reset_index(name='match_count')

# Get the top core rosters by the number of matches played
top_core_rosters = core_match_counts.sort_values(by='match_count', ascending=False)

# Merge to get player names for each core_id
merged_df = pd.merge(team_members_df, top_core_rosters, on='core_id')

# Group by core_id and aggregate unique player names
core_roster_with_names = merged_df.groupby('core_id')['name'].apply(lambda x: list(set(x))).reset_index()
core_roster_with_names['match_count'] = top_core_rosters['match_count'].values

# Sort by match_count to show the most matches played
core_roster_with_names = core_roster_with_names.sort_values(by='match_count', ascending=False)

# Display the result
print(core_roster_with_names)

      core_id                                               name  match_count
0           1  [MILO, SEAGULL, NUMLOCKED, OPLAID, DUMMY, CLOC...          967
1           2         [DOT, B1AM, VORPAL, NBLITZ, SPOH, EMPERIC]          835
2           3      [DAFOX, TVIQ, RENBOT, DEZU, KUDOCHOP, FAVION]          818
3           4      [REAVER, KYKY, ADAM, GREGO, SUREFOUR, DEBETT]          766
4           5  [MTG, KRAWNNIC, NAPTIME, MANGACHU, DUMPTRUCK, ...          764
...       ...                                                ...          ...
6490     6491       [YLDA, YUVAI, SA7AYB, SHAMMA, JAY, FNO, MAY]            2
6491     6492        [BEAU, DONUT, RURY, KDO, WESAL, POWA, RUDY]            2
6492     6493  [SENO, MARO, MYTHICAL, MAISA, LOTUS, ADRENALINDA]            1
6493     6494  [NADEN, YARAEN, BLANK, LUND, PASTA, RASEEL, CA...            1
6494     6495  [BAZZANELLA, JIRAIYA, LONELY, MEIASLUPPO, JEFE...            1

[6495 rows x 3 columns]


In [69]:
# Saving the DataFrame as a pickle file
tournament_info_df.to_pickle('tournament_info_df.pkl')
match_cards_df.to_pickle('match_cards_df.pkl')
maps_df.to_pickle('maps_df.pkl')
team_members_df.to_pickle('team_members_df.pkl')

In [70]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect('tournaments.db')

# Create cursor
cur = conn.cursor()

# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS tournament_info (
    tournament_id INTEGER PRIMARY KEY,
    name TEXT,
    series TEXT,
    organizer TEXT,
    type TEXT,
    city TEXT,
    country TEXT,
    prizepool TEXT,
    format TEXT,
    date TEXT,
    sdate TEXT,
    edate TEXT,
    team_number INTEGER,
    agg_date TEXT,
    liquipediatier FLOAT,
    date_order INTEGER,
    has_players BOOLEAN,
    version INTEGER
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS match_cards (
    match_card_id INTEGER PRIMARY KEY,
    tournament_id INTEGER,
    date TEXT,
    date_timezone TEXT,
    opponent1 TEXT,
    opponent2 TEXT,
    format TEXT,
    date_time TEXT,
    key TEXT,
    agg_date TEXT,
    clean_opponent1 TEXT,
    clean_opponent2 TEXT,
    opponent1_score INTEGER,
    opponent2_score INTEGER,
    winner INTEGER,
    date_order INTEGER,
    liquipediatier FLOAT,
    team1_id FLOAT,
    team2_id FLOAT,
    team1_avg_elo FLOAT,
    team2_avg_elo FLOAT,
    team1_expected_outcome FLOAT,
    team2_expected_outcome FLOAT,
    version INTEGER,
    FOREIGN KEY(tournament_id) REFERENCES tournament_info(tournament_id)
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS maps (
    match_card_id INTEGER,
    map TEXT,
    mode TEXT,
    map_num TEXT,
    score1 FLOAT,
    score2 FLOAT,
    winner FLOAT,
    FOREIGN KEY(match_card_id) REFERENCES match_cards(match_card_id)
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS team_members (
    unique_id INTEGER,
    team_id INTEGER,
    tournament_id INTEGER,
    name TEXT,
    position TEXT,
    flag TEXT,
    role_type TEXT,
    clean_team TEXT,
    elo_before FLOAT,
    elo_after FLOAT,
    c_elo_before FLOAT,
    c_elo_after FLOAT,
    core_id INTEGER,
    FOREIGN KEY(team_id) REFERENCES match_cards(team1_id),
    FOREIGN KEY(team_id) REFERENCES match_cards(team2_id),
    FOREIGN KEY(tournament_id) REFERENCES tournament_info(tournament_id)
)
''')

# Insert data into the tables, replacing existing tables if they already exist
tournament_info_df.to_sql('tournament_info', conn, if_exists='replace', index=False)
match_cards_df.to_sql('match_cards', conn, if_exists='replace', index=False)
maps_df.to_sql('maps', conn, if_exists='replace', index=False)
team_members_df.to_sql('team_members', conn, if_exists='replace', index=False)

# Create indexes
cur.execute('CREATE INDEX IF NOT EXISTS idx_team_members_team_id ON team_members(team_id);')
cur.execute('CREATE INDEX IF NOT EXISTS idx_match_cards_team1_id ON match_cards(team1_id);')
cur.execute('CREATE INDEX IF NOT EXISTS idx_match_cards_team2_id ON match_cards(team2_id);')
cur.execute('CREATE INDEX IF NOT EXISTS idx_match_cards_match_card_id ON match_cards(match_card_id);')
cur.execute('CREATE INDEX IF NOT EXISTS idx_maps_match_card_id ON maps(match_card_id);')

# Commit changes and close connection
conn.commit()
conn.close()